In [9]:
import pandas as pd
import re
import os
import chardet

# ===== 0) 경로 설정 =====
# 상대 경로로 입력 파일 설정
relative_data_path = '../../data/raw/'
file_name = 'Y06_서울시 안심귀갓길 안전시설물.csv'
input_file = os.path.join(relative_data_path, file_name)

# 출력 경로 및 파일 이름 설정
output_dir = '../../data/preprocessing'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(
    output_dir,
    'Y06_전처리_서울시 안심귀갓길 안전시설물.csv'
)

# ===== 1) 인코딩 자동 감지 및 파일 로드 =====
df = pd.read_csv(input_file, sep=",", encoding="cp949", engine="python")
print(f"\n✅ [1단계] 파일 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}열")
print("📌 원본 컬럼:", df.columns.tolist())
df.info()


✅ [1단계] 파일 로드 완료: 11,883행 × 18열
📌 원본 컬럼: ['포인트 wkt', '시설물 id', '시군구 코드', '시군구명', '읍면동 코드', '읍면동명', '시설코드', '안심귀갓길 아이디', '안심귀갓길 명', '설치대수', '비고', '관리기관', '전화번호', '조성년월', '시설물 최종점검일(사용안함)', '세부위치설명', '데이터 기준일자', '이미지명']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11883 entries, 0 to 11882
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   포인트 wkt          11883 non-null  object 
 1   시설물 id           11883 non-null  object 
 2   시군구 코드           11883 non-null  int64  
 3   시군구명             11883 non-null  object 
 4   읍면동 코드           11883 non-null  int64  
 5   읍면동명             11883 non-null  object 
 6   시설코드             11883 non-null  int64  
 7   안심귀갓길 아이디        11883 non-null  object 
 8   안심귀갓길 명          11883 non-null  object 
 9   설치대수             11883 non-null  int64  
 10  비고               1138 non-null   object 
 11  관리기관             4289 non-null   object 
 12  전화번호             4172 n

In [ ]:
# ===== 2) 구 추출 함수 =====
def extract_gu(gu_name):
    if pd.isna(gu_name):
        return None
    match = re.search(r"(\S+구)", str(gu_name))
    return match.group(1) if match else None

# ===== 3) 컬럼 생성 (구만) =====
df["구"] = df["시군구명"].apply(extract_gu)

# ===== 4) 시설코드 매핑 =====
facility_map = {
    301: "안심벨",
    302: "CCTV",
    303: "안내표지판(전주포함)",
    304: "노면표기",
    305: "보안등",
    306: "안심귀갓길 안내판",
    307: "112 위치신고 안내",
    308: "기타"
}
df["시설종류"] = df["시설코드"].map(facility_map)

# ===== 5) 피벗으로 시설별 설치 수 집계 (구 단위) =====
df_pivot = pd.pivot_table(
    df,
    index=["구"],
    columns="시설종류",
    values="시설코드",
    aggfunc="count",
    fill_value=0
).reset_index()

# ===== 6) 컬럼명 변경 =====
rename_map = {
    "구": "지역구",
    "안심벨": "안심벨 설치수",
    "CCTV": "CCTV 설치수",
    "안내표지판(전주포함)": "안내표지판 설치수",
    "노면표기": "노면표기 설치수",
    "보안등": "보안등 설치수",
    "안심귀갓길 안내판": "안심귀갓길 서비스 안내판 설치수",
    "112 위치신고 안내": "112 위치 신고 안내 설치수",
    "기타": "기타 (안심반사경 등) 설치수"
}
df_pivot = df_pivot.rename(columns=rename_map)

# ===== 7) 지역구 기준 오름차순 정렬 =====
df_pivot = df_pivot.sort_values(by=["지역구"], ascending=True).reset_index(drop=True)

# ===== 8) 결측치 확인 =====
cols_to_check = ["지역구", "CCTV 설치수", "안심벨 설치수"]
missing_counts = df_pivot[cols_to_check].isna().sum()
print("\n🔍 지정 컬럼 결측치:\n", missing_counts)

# ===== 9) CCTV만 추출해 저장 =====
cctv_df = df_pivot[["지역구", "CCTV 설치수"]]
cctv_file = os.path.join(output_dir, "Y06_전처리_CCTV_설치수_20230421기준.csv")
cctv_df.to_csv(cctv_file, index=False, encoding="utf-8-sig")
print(f"\n💾 CCTV 결과 저장 완료: {cctv_file}")

# ===== 10) 안심벨만 추출해 저장 =====
bell_df = df_pivot[["지역구", "안심벨 설치수"]]
bell_file = os.path.join(output_dir, "Y06_전처리_안심벨_설치수_20230421기준.csv")
bell_df.to_csv(bell_file, index=False, encoding="utf-8-sig")
print(f"💾 안심벨 결과 저장 완료: {bell_file}")

# ===== 11) 결과 확인 =====
print("\n✅ CCTV 결과 미리보기:")
print(cctv_df.head(10))

print("\n✅ 안심벨 결과 미리보기:")
print(bell_df.head(10))


🔍 지정 컬럼 결측치:
 시설종류
지역구         0
CCTV 설치수    0
안심벨 설치수     0
dtype: int64

💾 CCTV 결과 저장 완료: ../../data/preprocessing/Y06_전처리_서울시 안심귀갓길 안전시설물_CCTV.csv
💾 안심벨 결과 저장 완료: ../../data/preprocessing/Y06_전처리_서울시 안심귀갓길 안전시설물_안심벨.csv

✅ CCTV 결과 미리보기:
시설종류  지역구  CCTV 설치수
0     강남구       122
1     강동구        48
2     강북구        92
3     강서구        43
4     관악구       158
5     광진구        33
6     구로구        39
7     금천구        27
8     노원구        60
9     도봉구        35

✅ 안심벨 결과 미리보기:
시설종류  지역구  안심벨 설치수
0     강남구       89
1     강동구       46
2     강북구       77
3     강서구       34
4     관악구       49
5     광진구       30
6     구로구       24
7     금천구       22
8     노원구       46
9     도봉구       26
